In [ ]:
import logging
import os
import warnings

import matplotlib.pyplot as plt
import matplotlib.style as style
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow_hub as hub

from datetime import datetime
from keras.preprocessing import image
from PIL import Image
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import regularizers, optimizers
import pandas as pd
import numpy as np

In [ ]:
filePaths=[]
for dirname, yy, filenames in os.walk('../input/sample/sample/sample/images/'):
    for filename in filenames:
            filePaths.append(dirname+"/"+filename)
filePaths.__len__()

In [ ]:
filePaths[:10]

In [ ]:
filePaths.sort()


In [ ]:
filePaths[:10]

In [ ]:
df = pd.read_csv('../input/sample/sample/sample_labels.csv')
# df2=pd.read_csv('../input/datata/Useful.csv')
# df["Path"]=filePaths
df.head()

In [ ]:
df['Path']=filePaths
df.head()

In [ ]:
df.isna().sum()

In [ ]:
labels=[x.split("|") for x in list(df['Finding Labels'])]
labels[:10]

In [ ]:
unique={}
counter=0
for x in labels:
    for y in x:
        if y in unique:
            unique[y]+=1
        else:
            unique[y]=1
        counter+=1
ss=[[x,unique[x]] for x in unique]
ss=sorted(ss,key=lambda x:x[1])
print(ss)


In [ ]:
df['Labels'] = df['Finding Labels'].apply(lambda s: ["Diseased" if l not in ['No Finding'] else "Healthy" for l in str(s).split('|')])

df.head()


In [ ]:
df['bin']=df['Finding Labels'].apply(lambda s: ["Diseased" if l not in ['No Finding'] else "Healthy" for l in str(s).split('|')])
df['bin']=df['bin'].apply(lambda s:sorted(s) )
df['bin']=df['bin'].apply(lambda s:s[0] )
df.head()

In [ ]:
df['bin'].value_counts()

In [ ]:
mapping={}
counter=0
for s in ss[1:]:
    mapping[s[0]]=counter
    counter+=1
mapping

In [ ]:
mapper={ mapping[k]:k for k in mapping}
mapper

In [ ]:
vectorized=[]
labels=list(df['Labels'])
for x in labels:
    temp=[0]*14
    for y in x:
        ix=mapping[y]
        temp[ix]=1
    vectorized.append(temp)
vectorized[:10]    

In [ ]:
df['Label']=vectorized
df.head()
        


In [ ]:
df_used=df[['Image Index','Path','Labels','Label']]

df_used.to_csv('Useful.csv')
df_used.head()

In [ ]:
tfcat=[]
for x in vectorized:
    y=np.array(x).transpose()
    y=np.reshape(y,(14,1))
    tfcat.append(y)

tfcat[:2]

In [ ]:
df_used['Lab']=tfcat

In [ ]:
df_used.head()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df['Path'], df['bin'], test_size=0.1, random_state=42)
print("Number of images for training: ", len(x_train))
print("Number of images for validation: ", len(x_test))

In [ ]:
x_train=list(x_train)
x_test=list(x_test)
y_train=list(y_train)
y_test=list(y_test)

In [ ]:
def map2lab(arr):
    labels=[]
    counter=0
    for i in range(len(arr)):
        if arr[i]:
            labels.append(mapper[i])
    return labels

In [ ]:
x_train[:10]

In [ ]:
y_train[:10]

In [ ]:
IMG_SIZE = 256 # Specify height and width of image to match the input format of the model
CHANNELS = 3

In [ ]:
BATCH_SIZE = 256
AUTOTUNE = tf.data.experimental.AUTOTUNE # Adapt preprocessing and prefetching dynamically
SHUFFLE_BUFFER_SIZE = 1024 # Shuffle the training data by a chunck of 1024 observations

In [ ]:
datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,horizontal_flip=True, vertical_flip=True, rescale=1./255.,
    validation_split=0.0, dtype=None
)


In [ ]:
train_generator=datagen.flow_from_dataframe(dataframe=df, 
                                            x_col="Path", 
                                            y_col="Labels", 
                                            classes=['Pneumonia','Fibrosis','Edema','Emphysema','Cardiomegaly','Pleural_Thickening','Consolidation','Pneumothorax','Mass','Nodule','Atelectasis','Effusion','Infiltration','No Finding'],
                                            target_size=(256,256), 
                                            batch_size=512)

In [ ]:
base_model=tf.keras.applications.efficientnet.EfficientNetB4(
    include_top=False, weights='imagenet',
    input_shape=(256,256,3))


In [ ]:
base_model.trainable=False

In [ ]:
model = Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(14, activation='sigmoid'))
model.compile(optimizers.Adam(),loss="binary_crossentropy",metrics=[tf.keras.metrics.AUC(multi_label=True)])

In [ ]:
model.summary()

In [ ]:
model.fit(train_generator,epochs=10)

In [ ]:
df_used=df2
df_used['binary']=df['bin']
df_used.head()

In [ ]:
datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    validation_split=0.0, dtype=None
)


In [ ]:
train_generator=datagen.flow_from_dataframe(dataframe=df, 
                                            x_col="Path", 
                                            y_col="bin", 
                                            class_mode='binary',
                                            target_size=(256,256), 
                                            batch_size=256)

In [ ]:
a=train_generator.next()


In [ ]:
x_train,y_train=a


In [ ]:
x_train[2].shape

In [ ]:
base_model=tf.keras.applications.efficientnet.EfficientNetB4(
    include_top=False, weights='imagenet',
    input_shape=(256,256,3))
base_model.trainable=False

In [ ]:
base_model.trainable=True

In [ ]:
model = Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizers.Adam(),loss="binary_crossentropy",metrics=[tf.keras.metrics.AUC()])

In [ ]:
model.summary()

In [ ]:
model = Sequential()

nb_filters = 32
kernel_size = (2,2)
model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1]),
                     padding='valid',
                     strides=1,
                     input_shape=(256, 256, 3)))
model.add(Activation('relu'))

model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
model.add(Activation('relu'))

model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

nb_filters = 64
kernel_size = (4, 4)

model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
model.add(Activation('relu'))

model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

nb_filters = 128
kernel_size = (8, 8)

model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
model.add(Activation('relu'))

model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(12, 12)))

model.add(Flatten())
print("Model flattened out to: ", model.output_shape)

model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(4096))
model.add(Activation("relu"))
model.add(Dropout(0.2))

model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(optimizers.Adam(),loss="binary_crossentropy",metrics=[tf.keras.metrics.AUC()])

In [ ]:
model.summary()

In [ ]:
model.fit(train_generator,epochs=100)

In [ ]:
model.save("Trained.h5")

In [ ]:
#loss: 0.7231 - auc_3: 0.5281

In [ ]:
nobs = 20 # Maximum number of images to display
ncols = 5 # Number of columns in display
nrows = nobs//ncols # Number of rows in display
import random
import cv2
style.use("default")
plt.figure(figsize=(12,4*nrows))
for i in range(nrows*ncols):
    k=random.randint(0,400)
    ax = plt.subplot(nrows, ncols, i+1)
    plt.imshow(Image.open(x_train[k]))
    aaaa=cv2.imread(x_train[k])
    aaaa=np.resize(aaaa,(1,256,256,3))
    aaaa=(1./255.)*(aaaa)
#     print(aaaa.shape)
    ter=model.predict(aaaa)
    
    ter=["Diseased" if ter<0.6 else "Healthy"]
    ter=ter[0]
    plt.title([(y_train[k]),ter], size=10)
    plt.axis('off')